# Demagnetization field

Let us assume a magnetization of the form
$$\mathbf{M}=M_{\mathsf{s}} \, \mathbf{m},$$
nonzero only inside a magnetic region $\Omega_1$.

This magnetization creates a magnetic field $\mathbf{H}^{\mathsf{dem}}$ on $\mathbb{R}^3$ satisfying
$$
\nabla \cdot (\mathbf{H}^{\mathsf{dem}} + \mathbf{M}) = 0.
$$

The magnetic field also satisfies
$$
\nabla \times \mathbf{H}^{\mathsf{dem}} = 0,
$$
thus it is a conservative field and admits a scalar potential $u$ satisfying
$$
\mathbf{H}^{\mathsf{dem}} = - \nabla u.
$$

The scalar field $u$ satisfies the Poisson's equation
$$
\Delta u = \nabla \cdot \mathbf{M}
$$
with open boundary conditions (i.e. $u$ is zero at infinity).

In this notebook we will evaluate the demagnetization field for $\Omega_m$ a single grain cube of side length 50, spontaneous magnetization $M_{\mathsf{s}}=1.4 \times 10^6 \mathrm{A}/\mathrm{m}$, and constant magnetization $\mathbf{m} (\mathbf{x}) = (0,0,1)$. We will however keep a general framework, that can be easily generalizable.

## Truncation approach and weak formulation

To solve this problem we use the so-called truncation approach, or airbox method.

We define a domain $\Omega$ big enough (such that $\Omega_1 \subset \Omega$ and $\Omega_2 := \Omega -\Omega_1$ is nonempty) and look for $u$ solution of
$$
\left\{
\begin{array}{ll}
    - \Delta u = - \nabla \cdot M_{\mathsf{s}} \, \mathbf{m} & \text{in } \Omega, \\
    u = 0 & \text{on } \partial \Omega.
\end{array}\right.
$$
where
$$
M_{\mathsf{s}} (\mathbf{x}) := \left\{
\begin{array}{ll}
    M_{\mathsf{s}}^1 := 1.4 \times 10^6 \ \mathrm{A}/\mathrm{m} & \mathbf{x} \in \Omega_1, \\
    M_{\mathsf{s}}^2 := 0 & \mathbf{x} \in \Omega_2.
\end{array}\right.
$$

We define the solution space
$$
V_0 := \{ v \in H^1(\Omega) : \, \left.v\right|_{\partial\Omega}=0 \}.
$$

Then the weak formulation of the problem reads: find $u \in V_0$ such that
$$
\int_\Omega \nabla u (\mathbf{x}) \cdot \nabla v (\mathbf{x}) \ \mathrm{d}\mathbf{x} = \sum_i M_{\mathsf{s}}^i \int_{\Omega_i} \mathbf{m} \cdot \nabla v (\mathbf{x}) \ \mathrm{d}\mathbf{x} \qquad \forall \, v \in V_0.
$$

## Loading the mesh

In [1]:
import skfem
from skfem.helpers import grad, dot, ddot
from skfem.models.poisson import mass
from skfem.visuals.matplotlib import draw
import numpy as np

In [2]:
mesh = skfem.Mesh.load("mesh/Omega.msh")
mesh.subdomains.pop("gmsh:bounding_entities", None)
mesh

<skfem MeshTet1 object>
  Number of elements: 291708
  Number of vertices: 48922
  Number of nodes: 48922
  Named subdomains [# elements]: core [71373], air [220335]
  Named boundaries [# facets]: boundary [8862]

In [3]:
# draw(mesh)

## Choosing parameters

We choose size (a scaling parameter) in such a way to describe an object in the nanometer-size.

In [4]:
size = 1e-9

Next, we choose the spontaneous magnetization for the two domains: the magnetic material (tagged `core`) and the non-magnetic material (tagged `air`).

In [5]:
Ms_values = {
    "core": 1.4e6,
    "air": 0,
}

We also define the magnetization $\mathbf{m} (\mathbf{x})$ as a constant (0,0,1) vector.

In [6]:
m_func = lambda x: np.array(
    [
        0 * x[0],
        0 * x[1],
        1 + 0*x[2]
    ]
)

## Discretizing the PDE

We can define elements on the whole mesh and also on specific subdomains. We can also change the type of elements for a specific basis.

In [7]:
basis_1 = skfem.Basis(mesh, skfem.ElementTetP1())
basis_0 = basis_1.with_element(skfem.ElementTetP0())  # change type of elements
basis_core = basis_1.with_elements(elements=mesh.subdomains['core'])  # change subdomain
basis_air = basis_1.with_elements(elements=mesh.subdomains['air'])

The left hand side can be discretized by assembling a `BilinearForm`.

In [8]:
@skfem.BilinearForm
def stiffness(u, v, _):
    return dot(grad(u), grad(v))
S = stiffness.assemble(basis_1) * size

For the right hand side there are two methods (in the general case with multiple subdomains): in the first one we use the fact that $M_{\mathsf{s}}$ is constant in each domain, while in the second one we describe a more general case that can be useful in some cases.

### RHS: Method 1

In the first method for assembling the right hand side we follow the formula
$$
\sum_i M_{\mathsf{s}}^i \int_{\Omega_i} \mathbf{m} (\mathbf{x}) \cdot \nabla v (\mathbf{x}) \ \mathrm{d}\mathbf{x}.
$$
We assemble a matrix for each domain: 

This is done by projecting the `m_func` defined previously into a vectorial function space (see the use of `ElementVector` later).

Once we project the magnetization function `m_func` into the vectorial space (in the subdomain parametrized by the variable `dom`), we obtain `m_dom`, a flattened array containing all entries. To retrieve a `DiscreteField` object with more information we use the `interpolate` function.

> Note: projection takes a while, as the value of the function on each quadrature point must be evaluated. For more details, see [here](https://scikit-fem.readthedocs.io/en/stable/extended.html).

The vectorial space is defined using

In [9]:
basis_V1 = skfem.Basis(mesh, skfem.ElementVector(skfem.ElementTetP1()))

As the linear form is unchanged (we only change the domains where we integrate), we define it once:

In [10]:
@skfem.LinearForm
def B(v, w):
    return dot(w.m, grad(v))

In [11]:
%%time
b = 0
for dom in ["core", "air"]:
    basis_dom_scalar = basis_1.with_elements(mesh.subdomains[dom])
    basis_dom_vectorial = basis_V1.with_elements(mesh.subdomains[dom])
    m_dom = basis_dom_vectorial.project(m_func)
    m_DF_dom = basis_dom_vectorial.interpolate(m_dom)
    b += Ms_values[dom] * B.assemble(
        basis_dom_scalar,
        m=m_DF_dom,
    )
b *= size**2

CPU times: user 1min 43s, sys: 3.44 s, total: 1min 47s
Wall time: 1min 47s


### RHS: Method 2

The second method assembles the right hand side directly from the linear functional
$$
B (v) = \int_\Omega M_{\mathsf{s}} (\mathbf{x}) \ \mathbf{m}(\mathbf{x}) \cdot \nabla v (\mathbf{x}) \ \mathrm{d}\mathbf{x} = \int_\Omega \mathbf{M} (\mathbf{x}) \cdot \nabla v (\mathbf{x}) \ \mathrm{d}\mathbf{x}
$$

Namely, we first multiply the field $\mathbf{m} (\mathbf{x})$ with the scalar function $M_\mathsf{s} (\mathbf{x})$ into a magnetization field $\mathbf{M}$.

We achieve this by creating the `DiscreteField` `Ms_DF`, a cell function (because constant on each cell).

In [12]:
Ms_array = basis_0.zeros()
for subdomain in mesh.subdomains:
    Ms_array[basis_0.get_dofs(elements=subdomain)] = Ms_values[subdomain]
Ms_DF = basis_0.interpolate(Ms_array)
Ms_DF

<skfem DiscreteField object>
  Quadrature points per element: 4
  Number of elements: 291708
  Order: 0
  Attributes: grad

Then, we evaluate the magnetization field $\mathbf{M}$ as a `DiscreteField` `M`. It is a little complex because the two factors live in different function spaces (one is vectorial and one is scalar).

In [13]:
%%time
m_DF = basis_V1.interpolate(basis_V1.project(m_func))

CPU times: user 14min 58s, sys: 7.2 s, total: 15min 5s
Wall time: 4min 3s


In [14]:
M = np.einsum('ijk,jk->ijk', m_DF, Ms_DF)  # the first is a (3, #elem, 1) vector and the second is only (#elem) vector.

Then, the linear form is assembled over the full domain: the magnetization varies its magnitude depending on the domain.

In [15]:
@skfem.LinearForm
def B(v, w):
    return dot(w.mx, grad(v)[0]) + dot(w.my, grad(v)[1]) + dot(w.mz, grad(v)[2])
b2 = B.assemble(basis_1, mx=M[0], my=M[1], mz=M[2]) * size**2

On the positive side, in this case we only assemble the right hand side once. On the negative side, we require to know the different values of $M_\mathsf{s}$ before doing so, and this is in the general case **not** what we need.

## Solve the linear problem

The solution requires us to `condense` the system, i.e. eliminating the nodes corresponding to the boundary conditions (because they are fixed).

In [16]:
%%time
y = skfem.solve(*skfem.condense(S, b, D=mesh.boundary_nodes()))

CPU times: user 4min 21s, sys: 2.02 s, total: 4min 23s
Wall time: 1min 8s


The unpacking `*` reflects the following behavior (the next cell does exactly what the previous one does).

In [17]:
%%time
from scipy.sparse.linalg import spsolve

S_cnd, rhs_cnd, _, sol_indices = skfem.condense(A=S, b=b, D=mesh.boundary_nodes())
sol = spsolve(A=S_cnd, b=rhs_cnd)

CPU times: user 4min 15s, sys: 1.66 s, total: 4min 16s
Wall time: 1min 3s


`sol` is the solution only defined on some indices: if we had
```python
u = basis_1.zeros()
u[sol_indices] = sol
```
then we would have `y=u`.

In [18]:
sol.shape

(44489,)

In [19]:
y.shape

(48922,)

We can save the solution using the `save` method.

In [20]:
mesh.save("solution/u.vtk", point_data={"u": y})

## Calculate the $\mathbf{H}^\mathrm{dem}$ from the scalar potential

Project the gradient of the scalar potential to the vector function space on which magnetisation is defined.

In [21]:
%%time
H_dem = basis_V1.project(-grad(basis_1.interpolate(y))) / size

CPU times: user 15min 3s, sys: 7.92 s, total: 15min 10s
Wall time: 4min 15s


In [22]:
H_dem.shape

(146766,)

In [23]:
mesh.save("solution/H_dem.vtk", {"H": H_dem[basis_V1.nodal_dofs].T})

## Demagnetization energy

The energy connected to the demagnetization field is given by
$$
E^{\mathsf{dem}} = - \frac{\mu_0}{2} \int_\Omega M_\mathsf{s} (\mathbf{x}) \, \mathbf{m} (\mathbf{x}) \cdot \mathbf{H}^{\mathsf{dem}} (\mathbf{x}) \ \mathrm{d}\mathbf{x}.
$$
In this case, this becomes
$$
E^{\mathsf{dem}} = - \frac{\mu_0}{2} M^{\mathsf{core}}_\mathsf{s} \int_{\Omega_m} H^{\mathsf{dem}}_z (\mathbf{x}) \ \mathrm{d}\mathbf{x}.
$$

We evaluate `Hz`, i.e. the $z$-component of $H^{\mathsf{dem}}$ on the subdomain `core`:

In [24]:
Hz = basis_core.interpolate(H_dem[basis_V1.nodal_dofs][2])
Hz

<skfem DiscreteField object>
  Quadrature points per element: 4
  Number of elements: 71373
  Order: 0
  Attributes: grad

And we evaluate the formula

In [25]:
from scipy.constants import mu_0
@skfem.Functional
def integral(w):
    return w["Hz"]
E_dem = - 0.5 * mu_0 * Ms_values["core"] * integral.assemble(basis_core, Hz=Hz) * size**3
E_dem

4.84589198709108e-17

In [26]:
print(f"The demagnetisation energy density: {E_dem / (50 * size)**3} J/m^3")

The demagnetisation energy density: 387671.3589672863 J/m^3


## Solution with `mammos-mumag`

Compare demag energy with the solution evaluated with `mammos-mumag`. This will be inevitably different, as `mammos-mumag` uses a different calculation method than the truncation method, namely the shell method.

In [27]:
import mammos_entity as me
from mammos_mumag.materials import Materials
from mammos_mumag.parameters import Parameters
from mammos_mumag.simulation import Simulation
sim = Simulation(
    mesh="cube50_singlegrain_msize2",
    materials=Materials(
        domains=[
            {
                "theta": 0.0,
                "phi": 0.0,
                "K1": 0.0,
                "K2": 0.0,
                "Ms": me.Ms(Ms_values["core"]),
                "A": 0.0,
            },
            {
                "theta": 0.0,
                "phi": 0.0,
                "K1": 0.0,
                "K2": 0.0,
                "Ms": 0.0,
                "A": 0.0,
            },
            {
                "theta": 0.0,
                "phi": 0.0,
                "K1": 0.0,
                "K2": 0.0,
                "Ms": 0.0,
                "A": 0.0,
            },
        ],
    ),
    parameters=Parameters(
        size=size,
        scale=0,
        m_vect=[0, 0, 1],
        hstart=1,
        hfinal=-1,
        hstep=-0.2,
        h_vect=[0.01745, 0, 0.99984],
        mstep=0.4,
        mfinal=-1.2,
        tol_fun=1e-10,
        tol_hmag_factor=1,
        precond_iter=10,
    ),
)

In [28]:
%%time
sim.run_hmag(outdir="mammos-mumag-hmag", name="cube")

CPU times: user 271 ms, sys: 259 ms, total: 529 ms
Wall time: 50.8 s


In [29]:
import pandas as pd
pd.read_csv("mammos-mumag-hmag/cube.csv", skiprows=1)

,name,value,explanation
0,E_field,399196.443355,Energy density evaluated from field (J/m^3).
1,E_gradient,399196.443355,Energy density evaluated from gradient (J/m^3).
2,E_analytic,410501.440570,Energy density evaluated analytically (J/m^3).
